In [1]:
import warnings
warnings.filterwarnings("ignore")

import ast
import os
import json
import pandas as pd
import torch
import importlib
import cv2 

from shutil import copyfile
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.model_selection import GroupKFold
from PIL import Image
from string import Template
from IPython.display import display

TRAIN_PATH = 'dataset'

ModuleNotFoundError: No module named 'torch'

In [2]:
# check Torch and CUDA version
print(f"Torch: {torch.__version__}")
!nvcc --version

NameError: name 'torch' is not defined

In [6]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_path(row):
    row['image_path'] = f'{TRAIN_PATH}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

In [7]:
df = pd.read_csv("/media/ken/01D7D016D3925700/kaggle/YOLOX/dataset/train.csv")
df.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]


In [8]:
# Taken only annotated photos
df["num_bbox"] = df['annotations'].apply(lambda x: str.count(x, 'x'))
df_train = df[df["num_bbox"]>0]

#Annotations 
df_train['annotations'] = df_train['annotations'].progress_apply(lambda x: ast.literal_eval(x))
df_train['bboxes'] = df_train.annotations.progress_apply(get_bbox)

#Images resolution
df_train["width"] = 1280
df_train["height"] = 720

#Path of images
df_train = df_train.progress_apply(get_path, axis=1)

AttributeError: 'Series' object has no attribute '_is_builtin_func'

In [6]:
kf = GroupKFold(n_splits = 7) 
df_train = df_train.reset_index(drop=True)
df_train['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train, y = df_train.video_id.tolist(), groups=df_train.sequence)):
    df_train.loc[val_idx, 'fold'] = fold

df_train.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,num_bbox,bboxes,width,height,image_path,fold
0,0,40258,16,16,0-16,"[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]",1,"[[559, 213, 50, 32]]",1280,720,dataset/train_images/video_0/16.jpg,5
1,0,40258,17,17,0-17,"[{'x': 558, 'y': 213, 'width': 50, 'height': 32}]",1,"[[558, 213, 50, 32]]",1280,720,dataset/train_images/video_0/17.jpg,5
2,0,40258,18,18,0-18,"[{'x': 557, 'y': 213, 'width': 50, 'height': 32}]",1,"[[557, 213, 50, 32]]",1280,720,dataset/train_images/video_0/18.jpg,5
3,0,40258,19,19,0-19,"[{'x': 556, 'y': 214, 'width': 50, 'height': 32}]",1,"[[556, 214, 50, 32]]",1280,720,dataset/train_images/video_0/19.jpg,5
4,0,40258,20,20,0-20,"[{'x': 555, 'y': 214, 'width': 50, 'height': 32}]",1,"[[555, 214, 50, 32]]",1280,720,dataset/train_images/video_0/20.jpg,5


In [7]:
HOME_DIR = '' 
DATASET_PATH = 'dataset/images'

!mkdir {HOME_DIR}dataset
!mkdir {HOME_DIR}{DATASET_PATH}
!mkdir {HOME_DIR}{DATASET_PATH}/train2017
!mkdir {HOME_DIR}{DATASET_PATH}/val2017
!mkdir {HOME_DIR}{DATASET_PATH}/annotations

mkdir: cannot create directory ‘dataset’: File exists
mkdir: cannot create directory ‘dataset/images’: File exists
mkdir: cannot create directory ‘dataset/images/train2017’: File exists
mkdir: cannot create directory ‘dataset/images/val2017’: File exists
mkdir: cannot create directory ‘dataset/images/annotations’: File exists


In [8]:
SELECTED_FOLD = 4

for i in tqdm(range(len(df_train))):
    row = df_train.loc[i]
    if row.fold != SELECTED_FOLD:
        copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/train2017/{row.image_id}.jpg')
    else:
        copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/val2017/{row.image_id}.jpg')

In [9]:
print(f'Number of training files: {len(os.listdir(f"{HOME_DIR}{DATASET_PATH}/train2017/"))}')
print(f'Number of validation files: {len(os.listdir(f"{HOME_DIR}{DATASET_PATH}/val2017/"))}')

Number of training files: 4284
Number of validation files: 635


In [10]:
def save_annot_json(json_annotation, filename):
    with open(filename, 'w') as f:
        output_json = json.dumps(json_annotation)
        f.write(output_json)

In [12]:
annotion_id = 0
def dataset2coco(df, dest_path):
    
    global annotion_id
    
    annotations_json = {
        "info": [],
        "licenses": [],
        "categories": [],
        "images": [],
        "annotations": []
    }
    
    info = {
        "year": "2021",
        "version": "1",
        "description": "COTS dataset - COCO format",
        "contributor": "",
        "url": "https://kaggle.com",
        "date_created": "2021-11-30T15:01:26+00:00"
    }
    annotations_json["info"].append(info)
    
    lic = {
            "id": 1,
            "url": "",
            "name": "Unknown"
        }
    annotations_json["licenses"].append(lic)

    classes = {"id": 0, "name": "starfish", "supercategory": "none"}

    annotations_json["categories"].append(classes)

    
    for ann_row in df.itertuples():
            
        images = {
            "id": ann_row[0],
            "license": 1,
            "file_name": ann_row.image_id + '.jpg',
            "height": ann_row.height,
            "width": ann_row.width,
            "date_captured": "2021-11-30T15:01:26+00:00"
        }
        
        annotations_json["images"].append(images)
        
        bbox_list = ann_row.bboxes
        
        for bbox in bbox_list:
            b_width = bbox[2]
            b_height = bbox[3]
            
            # some boxes in COTS are outside the image height and width
            if (bbox[0] + bbox[2] > 1280):
                b_width = bbox[0] - 1280 
            if (bbox[1] + bbox[3] > 720):
                b_height = bbox[1] - 720 
                
            image_annotations = {
                "id": annotion_id,
                "image_id": ann_row[0],
                "category_id": 0,
                "bbox": [bbox[0], bbox[1], b_width, b_height],
                "area": bbox[2] * bbox[3],
                "segmentation": [],
                "iscrowd": 0
            }
            
            annotion_id += 1
            annotations_json["annotations"].append(image_annotations)
        
        
    print(f"Dataset COTS annotation to COCO json format completed! Files: {len(df)}")
    return annotations_json

In [12]:
df_train[df_train.fold != SELECTED_FOLD]

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,num_bbox,bboxes,width,height,image_path,fold
0,0,40258,16,16,0-16,"[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]",1,"[[559, 213, 50, 32]]",1280,720,dataset/train_images/video_0/16.jpg,5
1,0,40258,17,17,0-17,"[{'x': 558, 'y': 213, 'width': 50, 'height': 32}]",1,"[[558, 213, 50, 32]]",1280,720,dataset/train_images/video_0/17.jpg,5
2,0,40258,18,18,0-18,"[{'x': 557, 'y': 213, 'width': 50, 'height': 32}]",1,"[[557, 213, 50, 32]]",1280,720,dataset/train_images/video_0/18.jpg,5
3,0,40258,19,19,0-19,"[{'x': 556, 'y': 214, 'width': 50, 'height': 32}]",1,"[[556, 214, 50, 32]]",1280,720,dataset/train_images/video_0/19.jpg,5
4,0,40258,20,20,0-20,"[{'x': 555, 'y': 214, 'width': 50, 'height': 32}]",1,"[[555, 214, 50, 32]]",1280,720,dataset/train_images/video_0/20.jpg,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4843,2,22643,6488,1125,2-6488,"[{'x': 194, 'y': 661, 'width': 30, 'height': 37}]",1,"[[194, 661, 30, 37]]",1280,720,dataset/train_images/video_2/6488.jpg,3
4844,2,22643,6489,1126,2-6489,"[{'x': 189, 'y': 662, 'width': 30, 'height': 37}]",1,"[[189, 662, 30, 37]]",1280,720,dataset/train_images/video_2/6489.jpg,3
4845,2,22643,6490,1127,2-6490,"[{'x': 185, 'y': 664, 'width': 30, 'height': 37}]",1,"[[185, 664, 30, 37]]",1280,720,dataset/train_images/video_2/6490.jpg,3
4846,2,22643,6491,1128,2-6491,"[{'x': 179, 'y': 677, 'width': 30, 'height': 37}]",1,"[[179, 677, 30, 37]]",1280,720,dataset/train_images/video_2/6491.jpg,3


In [13]:
# Convert COTS dataset to JSON COCO
train_annot_json = dataset2coco(df_train[df_train.fold != SELECTED_FOLD], f"{HOME_DIR}{DATASET_PATH}/train2017/")
val_annot_json = dataset2coco(df_train[df_train.fold == SELECTED_FOLD], f"{HOME_DIR}{DATASET_PATH}/val2017/")

# Save converted annotations
save_annot_json(train_annot_json, f"{HOME_DIR}{DATASET_PATH}/annotations/train.json")
save_annot_json(val_annot_json, f"{HOME_DIR}{DATASET_PATH}/annotations/valid.json")

Dataset COTS annotation to COCO json format completed! Files: 4284
Dataset COTS annotation to COCO json format completed! Files: 635


In [16]:
# train_annot_json

In [17]:
# val_annot_json

In [13]:
config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 1
        self.width = 1
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        self.num_classes = 1
        
        self.data_dir = "dataset/images"
        self.train_ann = "train.json"
        self.val_ann = "valid.json"
        
        self.print_interval = 100
        self.eval_interval = 1
        self.data_num_workers = 4
        self.input_size = (800, 1280)
        self.test_size = (800, 1280)

'''

In [14]:
PIPELINE_CONFIG_PATH='cots_config.py'

pipeline = Template(config_file_template).substitute(max_epoch = 25)

with open(PIPELINE_CONFIG_PATH, 'w') as f:
    f.write(pipeline)

In [15]:
# ./yolox/data/datasets/voc_classes.py

voc_cls = '''
VOC_CLASSES = (
  "starfish",
)
'''
with open('./yolox/data/datasets/voc_classes.py', 'w') as f:
    f.write(voc_cls)

# ./yolox/data/datasets/coco_classes.py

coco_cls = '''
COCO_CLASSES = (
  "starfish",
)
'''
with open('./yolox/data/datasets/coco_classes.py', 'w') as f:
    f.write(coco_cls)


In [16]:
!cp ./tools/train.py ./

In [17]:
!python train.py \
    -f cots_config.py \
    -d 1 \
    -b 4 \
    --fp16 \
    -o \
    -c cots_pretrain/best_ckpt.pth

2021-12-30 14:53:27 | INFO     | yolox.core.trainer:126 - args: Namespace(batch_size=4, cache=False, ckpt='cots_pretrain/best_ckpt.pth', devices=1, dist_backend='nccl', dist_url=None, exp_file='cots_config.py', experiment_name='cots_config', fp16=True, machine_rank=0, name=None, num_machines=1, occupy=True, opts=[], resume=False, start_epoch=None)
2021-12-30 14:53:27 | INFO     | yolox.core.trainer:127 - exp value:
╒══════════════════╤═══════════════════╕
│ keys             │ values            │
╞══════════════════╪═══════════════════╡
│ seed             │ None              │
├──────────────────┼───────────────────┤
│ output_dir       │ './YOLOX_outputs' │
├──────────────────┼───────────────────┤
│ print_interval   │ 100               │
├──────────────────┼───────────────────┤
│ eval_interval    │ 1                 │
├──────────────────┼───────────────────┤
│ num_classes      │ 1                 │
├──────────────────┼───────────────────┤
│ depth            │ 1                 │
├──────